# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since you have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [2]:
df = pd.read_csv("Dataframes/characteristics_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Market Capitalization,Earnings Per Share – Coefficient of Variation,loss firm status
0,AVY.N,2013-01-01,Materials,2.499631,-0.378055,-0.392700,-0.170865,-1
1,AVY.N,2013-04-01,Materials,1.247607,-0.380857,-0.385490,-0.235084,-1
2,AVY.N,2013-07-01,Materials,0.726582,-0.376364,-0.385908,-0.255193,-1
3,AVY.N,2013-10-01,Materials,2.207725,-0.380352,-0.385784,-0.205410,-1
4,AVY.N,2014-01-01,Materials,0.904623,-0.373706,-0.380857,-0.187991,-1
...,...,...,...,...,...,...,...,...
18604,POOL.OQ,2021-10-01,Retailing,2.901092,-0.388214,-0.278490,-0.202958,-1
18605,POOL.OQ,2022-01-01,Retailing,3.720063,-0.419838,-0.235483,-0.185330,-1
18606,POOL.OQ,2022-04-01,Retailing,3.565072,-0.388113,-0.282188,-0.048679,-1
18607,POOL.OQ,2022-07-01,Retailing,0.917490,-0.334002,-0.305930,-0.170880,-1


Standardising and log transformation

#### Fixed Effects Panel Regression

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [3]:
#reformatting indices of dataframe for panel regression
df['Instrument'] = df['Instrument'].astype('category')
df['Date'] = pd.to_datetime(df['Date'])

# set the index to be the time variable and the cross-sectional variable
df.set_index(['Instrument', 'Date'], inplace=True)
#df_clean

In [4]:
df

GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
AVY.N      2013-01-01                                      2.499631   
           2013-04-01                                      1.247607   
           2013-07-01                                      0.726582   
           2013-10-01                                      2.207725   
           2014-01-01                                      0.904623   
...                                                             ...   
POOL.OQ    2021-10-01                                      2.901092   
           2022-01-01                                      3.720063   
           2022-04-01                                      3.565072   
           2022-07-01                                      0.917490   
           2022-10-01                                      1.647889   

                       Revenue - Actual  Market Capitalization  \
Instrument Date                                                  
AVY.N      2013-01-01         -0.378055              -0.392700   
           2013-04-01         -0.380857              -0.385490   
           2013-07-01         -0.376364              -0.385908   
           2013-10-01         -0.380352              -0.385784   
           2014-01-01         -0.373706              -0.380857   
...                                 ...                    ...   
POOL.OQ    2021-10-01         -0.388214              -0.278490   
           2022-01-01         -0.419838              -0.235483   
           2022-04-01         -0.388113              -0.282188   
           2022-07-01         -0.334002              -0.305930   
           2022-10-01         -0.371061              -0.317844   

                       Earnings Per Share – Coefficient of Variation  \
Instrument Date                                                        
AVY.N      2013-01-01                                      -0.170865   
           2013-04-01                                      -0.235084   
           2013-07-01                                      -0.255193   
           2013-10-01                                      -0.205410   
           2014-01-01                                      -0.187991   
...                                                              ...   
POOL.OQ    2021-10-01                                      -0.202958   
           2022-01-01                                      -0.185330   
           2022-04-01                                      -0.048679   
           2022-07-01                                      -0.170880   
           2022-10-01                                      -0.161998   

                       loss firm status  
Instrument Date                          
AVY.N      2013-01-01                -1  
           2013-04-01                -1  
           2013-07-01                -1  
           2013-10-01                -1  
           2014-01-01                -1  
...                                 ...  
POOL.OQ    2021-10-01                -1  
           2022-01-01                -1  
           2022-04-01                -1  
           2022-07-01                -1  
           2022-10-01                -1  

[18609 rows x 6 columns]

Regression with **Absolute** Surprise Values

In [5]:
y = df.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0479
Estimator:,PanelOLS,R-squared (Between):,-0.0456
No. Observations:,18541,R-squared (Within):,0.0539
Date:,"Wed, Mar 08 2023",R-squared (Overall):,-0.0407
Time:,16:38:11,Log-likelihood,-2.349e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,226.57
Entities:,502,P-value,0.0000
Avg Obs:,36.934,Distribution:,"F(4,17996)"
Min Obs:,2.0000,,
Max Obs:,40.000,F-statistic (robust):,226.57


#### By Industry: Example Tech Industry

With Absolute Surprise Values

In [14]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Revenue - Actual":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_69082/3204827800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_69082/3204827800.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0281
Estimator:,PanelOLS,R-squared (Between):,-0.1188
No. Observations:,728,R-squared (Within):,0.0361
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-0.0972
Time:,20:19:03,Log-likelihood,-842.14
Cov. Estimator:,Unadjusted,,
,,F-statistic:,3.1984
Entities:,19,P-value,0.0042
Avg Obs:,38.316,Distribution:,"F(6,664)"
Min Obs:,27.000,,
Max Obs:,40.000,F-statistic (robust):,3.1984
